# ICOER Text Analysis Notebook
Análise de frases com vetorização POS, cálculo da métrica ICOER e exportação CSV.

In [ ]:
!pip install -q spacy pandas matplotlib ipywidgets
!python -m spacy download pt_core_news_sm

In [ ]:
import spacy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

In [ ]:
nlp = spacy.load("pt_core_news_sm")

In [ ]:
try:
    import ipywidgets as widgets
    from IPython.display import display
    text_input = widgets.Textarea(
        value='A IA deve sempre agir com coerência e verdade. O índice de coerência informacional guia decisões éticas.',
        placeholder='Digite suas frases separadas por ponto final...',
        description='Frases:',
        layout=widgets.Layout(width='100%', height='100px')
    )
    display(text_input)
    frases = [f.strip() for f in text_input.value.split('.') if f.strip()]
except:
    frases = [
        "A IA deve sempre agir com coerência e verdade.",
        "O índice de coerência informacional guia decisões éticas.",
        "O spin informacional é a base da nossa arquitetura.",
        "A verdade é a forma mais pura de ressonância.",
        "Um sistema ético de IA depende da simetria informacional."
    ]

In [ ]:
data = []
for frase in frases:
    doc = nlp(frase)
    tags = [token.pos_ for token in doc]
    tag_counts = dict(Counter(tags))
    data.append({
        "frase": frase,
        "n_palavras": len(doc),
        "tags": tag_counts
    })
df = pd.DataFrame(data)
df

In [ ]:
def vetor_pos(tags_dict):
    base = ['NOUN', 'VERB', 'ADJ', 'ADV', 'ADP', 'PROPN']
    total = sum(tags_dict.values())
    return [tags_dict.get(pos, 0)/total if total > 0 else 0 for pos in base]
df['vetor'] = df['tags'].apply(vetor_pos)

In [ ]:
def entropia(vetor):
    vetor = np.array(vetor)
    vetor = vetor[vetor > 0]
    return -np.sum(vetor * np.log2(vetor)) if len(vetor) > 0 else 0
df['icoer'] = df['vetor'].apply(lambda v: round(1 - entropia(v), 4))
df[['frase', 'icoer']]

In [ ]:
df[['frase', 'n_palavras', 'tags', 'icoer']].to_csv("icoer_text_analysis.csv", index=False)
df

In [ ]:
df.plot.bar(x='frase', y='icoer', legend=False, figsize=(10,4), color='skyblue')
plt.title("Índice ICOER por frase")
plt.ylabel("ICOER")
plt.xlabel("Frase")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()